In [ ]:
import seaborn as sns
import numpy as np
import arviz as az
import pandas as pd
from scipy import stats
from baynes import get_model, multithreaded_run

import cmdstanpy
import logging
cmdstanpy.utils.get_logger().setLevel(logging.ERROR)

# Simulation-Based-Calibration of poisson model

In [ ]:
model = get_model('poisson_SBC.stan')
print(model.code())

### Run the model many times with multithreading

In [ ]:
N_ranks = 200
thinning = 2

def run_SBC(model):
    data = {'N': 500, 'alpha':5, 'beta':1, 'alpha_true':5, 'beta_true':1}
    fit = model.sample(data,
                         chains=1,
                         iter_warmup=200,
                         iter_sampling= (N_ranks - 1) * thinning,
                         save_warmup=False,
                         show_progress=False,
                         thin=thinning)
    df = fit.draws_pd('lt_lambda')
    rank = np.sum(df.to_numpy())
    return rank

rank = multithreaded_run(run_SBC, [model]*400, 4)

### ECDF difference plot

In [ ]:
pdf = stats.uniform(0,N_ranks)
ax = az.plot_ecdf(rank, cdf = pdf.cdf, difference=True,  plot_kwargs={'alpha':1, 'zorder':5}, fill_kwargs={'color': 'grey'})
ax.set_xlabel('rank')
ax.set_ylabel('ECDF difference')
ax.plot([0]*200, ls='--', color='black', alpha=0.4, zorder=0)

### Histogram and $\chi^2$ test

In [ ]:
def SBC_histogram(data, n_bins, percs=[0.05, 0.95]):
    if isinstance(data, str):
        data = pd.read_json(data)
    elif isinstance(data, dict):
        data = pd.DataFrame.from_dict(data)
    hist = sns.displot(data.melt(value_name='rank'), bins=n_bins, kind='hist', x='rank', col='variable', col_wrap=2, alpha=1)
    pdf = stats.binom(len(data), 1/n_bins)
    hist.set_titles("")
    for ax in hist.axes.flatten():
        xlim = ax.get_xlim()
        ax.fill_between(ax.get_xlim(), pdf.ppf(percs[0]), pdf.ppf(percs[1]), color='grey', alpha=0.20, zorder=0)
        ax.axhline(pdf.median(), color='grey', alpha=0.50, zorder=0)
        ax.set_xlim(xlim)
        ax.grid(visible=False)
    return hist

In [ ]:
N_bins = 20
hist = SBC_histogram({'lambda':rank}, N_bins)
exp = len(rank)/N_bins
counts, bins=np.histogram(rank, bins=N_bins)
chi_test = sum([(a-exp)**2/exp for a in counts])
p_value = stats.chi2.sf(chi_test, N_bins)
print("p-value: ", p_value)